In [ ]:
import gradio as gr
import cv2
import numpy as np
import time
from PIL import Image as PILImage
from ultralytics import YOLO
from transformers import CLIPProcessor, CLIPModel
from gtts import gTTS 
import random
import os

gr.close_all()
# --- LÓGICA DEL SISTEMA (BACKEND) ---
class GymkhanaMaster:
    def __init__(self):
        print("🛠️ System Init: Cargando Modelos Neurales...")
        self.yolo = YOLO('yolov8n.pt')
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.current_target = None
        self.time_limit = 60
        self.start_time = 0
        self.game_active = False
        self.modo_historia = "Agente Especial"

    def _hablar(self, texto):
        """Genera el archivo de audio con la misión"""
        try:
            # Usamos lang='en' para que pronuncie bien el objeto en inglés
            # o 'es' si prefieres que la intro sea en español aunque el objeto sea inglés.
            # Aquí uso 'es' para mantener la coherencia con la interfaz, 
            # asumiendo que el acento "spanglish" es aceptable para la demo.
            tts = gTTS(text=texto, lang='es')
            filename = "audio_mision.mp3"
            tts.save(filename)
            return filename
        except Exception as e:
            print(f"Error TTS: {e}")
            return None

    def configurar_partida(self, dificultad, modo):
        tiempos = {"🟢 Recluta (60s)": 60, "🟡 Veterano (30s)": 30, "🔴 Élite (15s)": 15}
        self.time_limit = tiempos.get(dificultad, 60)
        self.modo_historia = modo
        return self.time_limit

    def generar_narrativa(self, objeto):
        """Genera HTML visual y el texto para el TTS"""
        obj_upper = objeto.upper() # El objeto estará en INGLÉS (ej: BOTTLE)
        
        # Icono y Texto según el modo de juego
        icono = "🎯"
        if "Zombie" in self.modo_historia: icono = "🧟"
        elif "Espía" in self.modo_historia: icono = "🕵️‍♂️"

        # Texto para el audio (Voz)
        # Nota: Al dejar el objeto en inglés, la voz dirá "Necesitas un BOTTLE"
        if "Zombie" in self.modo_historia:
            voz = f"Superviviente. Necesitas encontrar el objeto: {objeto}. Tienes {self.time_limit} segundos."
            subtitulo = "⚠️ SUMINISTRO VITAL REQUERIDO"
        elif "Espía" in self.modo_historia:
            voz = f"Agente. Intercepte el objetivo: {objeto}, inmediatamente."
            subtitulo = "📁 MISIÓN CLASIFICADA: RECUPERACIÓN"
        else:
            voz = f"Misión iniciada. Encuentra el objeto: {objeto}. El tiempo corre."
            subtitulo = "🎯 OPERACIÓN ESTÁNDAR"

        # HTML PARA LA PANTALLA (VISUAL GIGANTE)
        html_visual = f"""
        <div class="hud-box">
            <div class="hud-sub">{icono} {subtitulo}</div>
            <div class="hud-target">{obj_upper}</div>
            <div class="hud-desc">Localice y asegure el objetivo antes de que acabe el tiempo.</div>
        </div>
        """
        
        return html_visual, voz

    def estampar_sello(self, imagen_numpy, texto_sello, es_victoria=True):
        """Estampa un sello visual sobre la imagen final"""
        img = imagen_numpy.copy()
        h, w = img.shape[:2]
        overlay = img.copy()
        color = (0, 255, 0) if es_victoria else (255, 0, 0)
        texto_sello = texto_sello.upper()
        font = cv2.FONT_HERSHEY_DUPLEX
        scale = 1.0 if w < 400 else 2.5
        thickness = 2 if w < 400 else 5
        (tw, th), _ = cv2.getTextSize(texto_sello, font, scale, thickness)
        cx, cy = w // 2, h // 2
        cv2.rectangle(overlay, (cx - tw//2 - 20, cy - th - 20), (cx + tw//2 + 20, cy + 20), color, thickness)
        cv2.putText(overlay, texto_sello, (cx - tw//2, cy), font, scale, color, thickness)
        alpha = 0.8
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
        return img

    def escanear_sala(self, imagen_sala):
        if imagen_sala is None: 
            return '<div class="hud-box-empty">ESPERANDO ESCANEO...</div>', None, None
        
        # 1. YOLO detecta objetos (En Inglés Nativo)
        pil_img = PILImage.fromarray(imagen_sala)
        cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        res = self.yolo(cv_img, conf=0.25, verbose=False)[0]
        objs = list(set([self.yolo.names[int(b.cls[0])] for b in res.boxes]))
        
        if not objs: 
            return '<div class="hud-box-error">⚠️ NO SE DETECTAN OBJETIVOS</div>', None, None

        # 2. Selección del objetivo
        self.current_target = random.choice(objs) # Ej: "bottle"
        self.start_time = time.time()
        self.game_active = True
        
        # 3. Generación de Narrativa y Audio
        html_mision, txt_voz = self.generar_narrativa(self.current_target)
        ruta_audio = self._hablar(txt_voz) # <--- AQUÍ SE GENERA EL TTS
        
        # 4. Barra de tiempo
        html_barra = f"""
        <div class="timer-container">
            <div class="timer-bar" style="animation-duration: {self.time_limit}s;"></div>
            <div class="timer-text">TIEMPO RESTANTE</div>
        </div>
        """
        # Devolvemos el path del audio para que Gradio lo reproduzca
        return html_mision, ruta_audio, html_barra

    def verificar_jugada(self, imagen_usuario):
        if not self.game_active: return "error", "SISTEMA INACTIVO", "Inicie escaneo.", None
        if imagen_usuario is None: return "error", "SIN SEÑAL", "Active cámara.", None

        pil_img = PILImage.fromarray(imagen_usuario)
        # CLIP trabaja nativamente en inglés, así que perfecto
        opciones = [f"a photo of a {self.current_target}", "something else"]
        
        inputs = self.clip_processor(text=opciones, images=pil_img, return_tensors="pt", padding=True)
        probs = self.clip_model(**inputs).logits_per_image.softmax(dim=1)
        
        if probs.argmax().item() == 0 and probs[0][0].item() > 0.6:
            self.game_active = False
            self._hablar(f"¡Objetivo {self.current_target} confirmado!")
            img_final = self.estampar_sello(imagen_usuario, "CONFIRMADO", True)
            return "win", "MISIÓN CUMPLIDA", f"El activo '{self.current_target.upper()}' ha sido asegurado.", img_final
        else:
            self._hablar("Objetivo incorrecto.")
            return "retry", "ERROR DE IDENTIFICACIÓN", "El objeto no coincide.", None

    def check_tiempo(self):
        if self.game_active and (time.time() - self.start_time > self.time_limit):
            self.game_active = False
            self._hablar("Tiempo agotado. Operación cancelada.")
            return True
        return False

juego = GymkhanaMaster()

# --- DISEÑO VISUAL (CSS) ---
css_tactical = """
@import url('https://fonts.googleapis.com/css2?family=Rajdhani:wght@500;700&display=swap');
body { background-color: #0f172a; color: #e2e8f0; font-family: 'Rajdhani', sans-serif; }
.gradio-container { max-width: 1000px !important; margin: auto; }

.glass-panel {
    background: rgba(30, 41, 59, 0.7); backdrop-filter: blur(10px);
    border: 1px solid rgba(148, 163, 184, 0.2); border-radius: 15px; padding: 20px;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.5); margin-bottom: 15px;
}

/* HUD OBJETIVO */
.hud-box {
    text-align: center; border: 1px solid #3b82f6; 
    background: radial-gradient(circle, rgba(30,58,138,0.6) 0%, rgba(15,23,42,0.8) 100%);
    border-radius: 10px; padding: 15px;
    box-shadow: 0 0 15px rgba(59, 130, 246, 0.3);
}
.hud-box-empty { text-align: center; color: #64748b; padding: 20px; font-size: 1.5em; letter-spacing: 2px; }
.hud-box-error { text-align: center; color: #ef4444; padding: 20px; font-weight: bold; }
.hud-sub { color: #94a3b8; font-size: 0.9em; letter-spacing: 3px; margin-bottom: 5px; text-transform: uppercase; }
.hud-target { 
    font-size: 3.5em; font-weight: 700; color: #ffffff; 
    text-shadow: 0 0 20px #22d3ee; margin: 10px 0;
    text-transform: uppercase; letter-spacing: 2px;
}
.hud-desc { color: #cbd5e1; font-size: 1em; font-style: italic; }

.timer-container {
    width: 100%; height: 24px; background: #1e293b; border-radius: 12px; overflow: hidden; position: relative;
    border: 1px solid #334155; margin-top: 10px;
}
.timer-bar {
    height: 100%; background: linear-gradient(90deg, #22d3ee, #3b82f6);
    width: 0%; animation-name: countdown; animation-timing-function: linear; animation-fill-mode: forwards;
}
.timer-text { position: absolute; top: 0; width: 100%; text-align: center; line-height: 24px; color: white; font-size: 14px; font-weight: bold; text-shadow: 0 1px 2px black; }
@keyframes countdown { from { width: 100%; background: #22d3ee; } to { width: 0%; background: #ef4444; } }

.primary-btn { background: linear-gradient(135deg, #2563eb, #1d4ed8) !important; border: none; color: white; }
.scan-btn { background: #334155 !important; border: 1px solid #475569; color: #e2e8f0; }
.action-btn { background: linear-gradient(135deg, #dc2626, #b91c1c) !important; color: white; }
.feedback-error { color: #f87171; text-align: center; font-weight: bold; font-size: 1.2em; animation: shake 0.5s; }
@keyframes shake { 0% { transform: translate(1px, 1px); } 50% { transform: translate(-1px, -1px); } 100% { transform: translate(1px, 1px); } }
"""

# --- INTERFAZ ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="cyan", neutral_hue="slate"), css=css_tactical) as demo:
    
    timer = gr.Timer(1, active=False)
    
    # MENÚ
    with gr.Column(visible=True, elem_classes=["glass-panel"]) as p_menu:
        gr.Markdown("# OBGUESSER ", elem_classes=["main-title"])
        gr.Markdown("SISTEMA DE ENTRENAMIENTO DE INTELIGENCIA VISUAL", elem_classes=["sub-title"])
        with gr.Row():
            sel_dif = gr.Radio(["🟢 Recluta (60s)", "🟡 Veterano (30s)", "🔴 Élite (15s)"], label="DIFICULTAD", value="🟢 Recluta (60s)")
            sel_mod = gr.Radio(["🕵️ Agente Espía", "🧟 Superviviente Zombie", "🏠 Modo Estándar"], label="ESCENARIO", value="🕵️ Agente Espía")
        btn_start = gr.Button("INICIAR OPERACIÓN", variant="primary", elem_classes=["primary-btn"])

    # JUEGO
    with gr.Column(visible=False) as p_juego:
        
        with gr.Row(elem_classes=["glass-panel"]):
            with gr.Column(scale=3):
                html_obj = gr.HTML(value='<div class="hud-box-empty">ESPERANDO DATOS DE MISIÓN...</div>')
            with gr.Column(scale=2):
                html_bar = gr.HTML() 

        with gr.Row():
            with gr.Column(elem_classes=["glass-panel"]):
                gr.Markdown("### 📡 1. INTELIGENCIA")
                in_sala = gr.Image(sources=["upload"], type="numpy", height=250, show_label=False)
                btn_scan = gr.Button("ANALIZAR ENTORNO", elem_classes=["scan-btn"])
            
            with gr.Column(elem_classes=["glass-panel"]):
                gr.Markdown("### 📸 2. CAMPO")
                in_cam = gr.Image(sources=["webcam"], type="numpy", height=250, show_label=False)
                btn_ok = gr.Button("CONFIRMAR OBJETIVO", elem_classes=["action-btn"])
        
        html_fb = gr.HTML(value='<div class="feedback-neutral">SISTEMA EN ESPERA...</div>', elem_classes=["glass-panel"])
        aud_out = gr.Audio(autoplay=True, visible=False)

    # FINAL
    with gr.Column(visible=False, elem_classes=["glass-panel"]) as p_final:
        gr.Markdown("# 📁 INFORME DE MISIÓN", elem_classes=["main-title"])
        img_result = gr.Image(show_label=False, interactive=False, height=400)
        lbl_res_titulo = gr.Markdown(elem_classes=["sub-title"])
        lbl_res_desc = gr.Markdown(elem_classes=["feedback-neutral"])
        btn_reset = gr.Button("NUEVA OPERACIÓN", variant="primary", elem_classes=["primary-btn"])

    # --- WIRING (CONEXIONES) ---
    def start_game(d, m):
        juego.configurar_partida(d, m)
        return {
            p_menu: gr.Column(visible=False), p_juego: gr.Column(visible=True), p_final: gr.Column(visible=False),
            in_sala: None, in_cam: None, 
            html_obj: '<div class="hud-box-empty">SUBE UNA FOTO PARA RECIBIR ÓRDENES</div>', 
            html_bar: "", html_fb: "", timer: gr.Timer(active=False)
        }
    btn_start.click(start_game, [sel_dif, sel_mod], [p_menu, p_juego, p_final, in_sala, in_cam, html_obj, html_bar, html_fb, timer])

    def scan_room(img):
        # Escaneamos y obtenemos el audio
        h_mision, ruta_audio, h_bar = juego.escanear_sala(img)
        feedback = '<div style="text-align:center; color:#22d3ee">¡OBJETIVO ASIGNADO!</div>' if h_bar else '<div class="feedback-error">ERROR DE IMAGEN</div>'
        # Activamos el timer y devolvemos el audio al componente aud_out
        return h_mision, ruta_audio, h_bar, feedback, gr.Timer(active=bool(h_bar))
    
    # IMPORTANTE: Conectamos aud_out aquí para que suene
    btn_scan.click(scan_room, in_sala, [html_obj, aud_out, html_bar, html_fb, timer])

    def verify_obj(img):
        res, tit, msg, final_img = juego.verificar_jugada(img)
        if res == "retry":
            return {html_fb: f'<div class="feedback-error">{msg}</div>'}
        else:
            return {
                p_juego: gr.Column(visible=False), p_final: gr.Column(visible=True),
                img_result: final_img, lbl_res_titulo: f"## {tit}", lbl_res_desc: msg,
                timer: gr.Timer(active=False)
            }
    btn_ok.click(verify_obj, in_cam, [html_fb, p_juego, p_final, img_result, lbl_res_titulo, lbl_res_desc, timer])

    def time_check():
        if juego.check_tiempo():
            black_img = np.zeros((400, 600, 3), dtype=np.uint8)
            fail_img = juego.estampar_sello(black_img, "MISION FALLIDA", False)
            return {
                p_juego: gr.Column(visible=False), p_final: gr.Column(visible=True),
                img_result: fail_img, lbl_res_titulo: "## TIEMPO AGOTADO", lbl_res_desc: "El objetivo se ha perdido.",
                timer: gr.Timer(active=False)
            }
        return gr.update()
    timer.tick(time_check, outputs=[p_juego, p_final, img_result, lbl_res_titulo, lbl_res_desc, timer])
    btn_reset.click(lambda: {p_menu: gr.Column(visible=True), p_final: gr.Column(visible=False)}, outputs=[p_menu, p_final])

demo.launch()